In [68]:
import pandas as pd
from glob import glob
import os
from datetime import datetime as dt
import numpy as np
import subprocess
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO

In [69]:
# From Thinned Data (Per Sample); condense it further using this:
data_thinned="/moto/ziab/users/yk2840/Frog_workspace/HMM/Ancestry_nofilter_results/posterior/data_thinned_filter_frac.2_072919.txt"

test = pd.read_csv(data_thinned, header = None, sep='\t')



In [70]:
snp_thinned="/moto/ziab/users/yk2840/Frog_workspace/HMM/Ancestry_nofilter_results/posterior/snps_thinned_frac0.2.txt"
ac_panel="/moto/ziab/users/yk2840/Frog_workspace/HMM/Ancestry_nofilter_results/posterior/F8254.posterior"
outdir='/moto/ziab/users/yk2840/Frog_workspace/HMM/Ancestry_nofilter_results/posterior/'

snps = pd.read_csv(snp_thinned, header = None, sep='\t', names = 'i')
snps=snps.sort_values(by='i')
ac = pd.read_csv(ac_panel, header = None, sep='\t')
ac = ac.iloc[snps.iloc[:,0],0:2]

ac.to_csv(outdir+'chrom_pos_prior_across_sampleprune.csv', sep=',', index=False, header=False)



/moto/opt/anaconda3-5.3.1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [71]:
difffrac = .01
gen_only=test.drop(test.columns[0], axis=1)


central_markers = gen_only.values[:, 1:-1]
left_markers = gen_only.values[:, :-2]
right_markers = gen_only.values[:, 2:]


left_diff = sum(np.where((central_markers != left_markers) & (left_markers!="-") & (central_markers!="-"), 1, 0))
right_diff = sum(np.where((central_markers != right_markers) & (right_markers!="-") & (central_markers!="-"), 1, 0))

left_total = sum(np.where((central_markers != "-") & (left_markers!="-"), 1, 0))
right_total = sum(np.where((central_markers != "-") & (right_markers!="-"), 1, 0))

indices = np.where(((right_diff/right_total)>=difffrac) | ((left_diff/left_total) >=difffrac))

newdf = gen_only[gen_only.columns[indices]]
newdf.to_csv(outdir+'samples_post_across_sampleprune.csv', sep=',', index=False, header=False)


In [72]:
ac = np.array(ac)
ac1 = ac[indices, :]
ac1=pd.DataFrame(ac1[0])

ac1.to_csv(outdir+'chrom_pos_post_across_sampleprune.csv', sep=',', index=False, header=False)


In [73]:
# Add the column heading to the top:
ac1 = ac1.T
ac1.columns = newdf.columns

new_df_headers = pd.concat([ac1,newdf])
new_df_headers.to_csv(outdir+'sample_headers_post_across_sampleprune.csv', sep=',', index=False, header=False)


In [48]:
panel_file="/moto/ziab/users/yk2840/Frog_workspace/HMM/Ancestry_nofilter_results/ancestry_panel_0removed_panel.txt"
print('Making genetic map.')
df = pd.read_csv(panel_file, sep='\t', header = None) # read into pandas dataframe
df = df.iloc[snps.iloc[:,0]-1,[0,1,6]]
df = np.array(df)
df = pd.DataFrame(df[indices, :][0])
df.columns = ['chr', 'position_in_bp', 'morgans']
df['marker']=df['chr'].astype(str)+':'+df['position_in_bp'].astype(str) # make list of markers in format chrom:position
df['cM'] = df['morgans'].map(lambda morgans: morgans) #convert from morgans to cM
df['pos'] = df.groupby('chr')['cM'].cumsum() #get cumulative sum of centimorgans for each chromosome
df = df[['marker','chr','pos']] #subset only marker, chromosome, and pos (in cM)

df.to_csv(outdir+'/F2_gmap_20190729.csv', sep=',', index=False, header=True)





Making genetic map.
